In [1]:
from tensorflow.keras.layers import (Input, Conv2D, Flatten, MaxPooling2D,
                                     Dense, Dropout, Concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
import numpy as np
import os
import pickle

In [2]:
# # Uncomment this box if the file is in google colab
# path2root = "/content/drive/MyDrive/Projects/EyeTracker/"

In [3]:
# Uncomment this box if the file is in your local directory
path2root = "../../"

In [4]:
subjects_fol = "subjects/"
data_ibo_fol = "data-ibo/"
models_fol = "models/"
models_ibo_fol = "ibo/"
raw_fol = "raw/"
chosen_inputs = [0, 1, 2, 6, 7, 8, 9]

In [5]:
data_ibo_dir = path2root + subjects_fol + f"{1}/" + data_ibo_fol

with open(data_ibo_dir + "x1.pickle", "rb") as f:
    x1 = pickle.load(f)
with open(data_ibo_dir + "x2.pickle", "rb") as f:
    x2 = pickle.load(f)
with open(data_ibo_dir + "y.pickle", "rb") as f:
    y = pickle.load(f)

x2_chs_inp = x2[:, chosen_inputs]

In [6]:
input1 = Input(x1.shape[1:])
layer = Conv2D(16, (5, 5), (1, 1), "same", activation="relu")(input1)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(32, (5, 5), (1, 1), "same", activation="relu")(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Conv2D(64, (3, 3), (1, 1), activation="relu")(layer)
layer = MaxPooling2D((2, 2), (2, 2))(layer)

layer = Flatten()(layer)

layer = Dense(256, "relu")(layer)

input2 = Input(x2_chs_inp.shape[1:])
layer = Concatenate()([layer, input2])

layer = Dense(128, "relu")(layer)

layer = Dense(32, "relu")(layer)

layer = Dense(16, "relu")(layer)

layer = Dense(3 ,"relu")(layer)

output_layer = Dense(y.max() + 1, "softmax")(layer)

input_layers = [input1, input2]

model = Model(inputs=input_layers, outputs=output_layer)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics="acc")

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 44, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 44, 16)   416         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 24, 22, 16)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 24, 22, 32)   12832       max_pooling2d[0][0]              
______________________________________________________________________________________________

In [7]:
# plot_model(model, show_shapes=True)

In [9]:
models_dir = path2root + models_fol
if not os.path.exists(models_dir):
    os.mkdir(models_dir)

models_ibo_dir = models_dir + models_ibo_fol
if not os.path.exists(models_ibo_dir):
    os.mkdir(models_ibo_dir)

raw_dir = models_ibo_dir + raw_fol
if not os.path.exists(raw_dir):
    os.mkdir(raw_dir)

In [10]:
models_numbers = []
models_name = os.listdir(raw_dir)
if models_name:
    for model_name in models_name:
        model_num = int(model_name[5:6])
        models_numbers.append(model_num)
    max_num = max(models_numbers)
else:
    max_num = 0

model.save(raw_dir + f"model{max_num+1}")

INFO:tensorflow:Assets written to: ../../models/ibo/raw/model2\assets
